In [2]:
import logging
import argparse

import onnx
import yaml

from pycocotools.coco import COCO
from pycocotools.mask import iou, encode
import numpy as np
from torchvision import transforms
from PIL import Image
from onnx import numpy_helper
import os
import onnxruntime
import math

In [3]:
class Dataloader:
    def __init__(self, root, img_dir='val2017', \
            anno_dir='annotations/instances_val2017.json'):
        import json
        import os
        import numpy as np
        from pycocotools.coco import COCO
        from neural_compressor.experimental.metric.coco_label_map import category_map
        self.batch_size = 1
        self.image_list = []
        img_path = os.path.join(root, img_dir)
        anno_path = os.path.join(root, anno_dir)
        coco = COCO(anno_path)
        img_ids = coco.getImgIds()
        cat_ids = coco.getCatIds()
        for idx, img_id in enumerate(img_ids):
            img_info = {}
            bboxes = []
            labels = []
            ids = []
            img_detail = coco.loadImgs(img_id)[0]
            ids.append(img_detail['file_name'].encode('utf-8'))
            pic_height = img_detail['height']
            pic_width = img_detail['width']

            ann_ids = coco.getAnnIds(imgIds=img_id,catIds=cat_ids)
            anns = coco.loadAnns(ann_ids)
            for ann in anns:
                bbox = ann['bbox']
                if len(bbox) == 0:
                    continue
                bboxes.append([bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]])
                labels.append(category_map[ann['category_id']].encode('utf8'))
            img_file = os.path.join(img_path, img_detail['file_name'])
            if not os.path.exists(img_file) or len(bboxes) == 0:
                continue

            if filter and not filter(None, bboxes):
                continue
            label = [np.array([bboxes]), np.array([labels]), np.zeros((1,0)), np.array([img_detail['file_name'].encode('utf-8')])]
            with Image.open(img_file) as image:
                image = image.convert('RGB')
                image, label = self.preprocess((image, label))
            self.image_list.append((image, label))

    def __iter__(self):
        for item in self.image_list:
            yield item

    def preprocess(self, sample):
        image, label = sample
        ratio = 800.0 / min(image.size[0], image.size[1])
        image = image.resize((int(ratio * image.size[0]), int(ratio * image.size[1])), Image.BILINEAR)

        # Convert to BGR
        image = np.array(image)[:, :, [2, 1, 0]].astype('float32')

        # HWC -> CHW
        image = np.transpose(image, [2, 0, 1])

        # Normalize
        mean_vec = np.array([102.9801, 115.9465, 122.7717])
        for i in range(image.shape[0]):
            image[i, :, :] = image[i, :, :] - mean_vec[i]

        # Pad to be divisible of 32
        padded_h = int(math.ceil(image.shape[1] / 32) * 32)
        padded_w = int(math.ceil(image.shape[2] / 32) * 32)

        padded_image = np.zeros((3, padded_h, padded_w), dtype=np.float32)
        padded_image[:, :image.shape[1], :image.shape[2]] = image
        image = padded_image
        bboxes, str_labels,int_labels, image_ids = label
        bboxes = ratio * bboxes
        return image, (bboxes, str_labels, int_labels, image_ids)

In [4]:
model = onnx.load("models/MaskRCNN-12.onnx")

In [5]:
class Post:
    def __call__(self, sample):
        preds, labels = sample
        bboxes, classes, scores, _ = preds
        bboxes = np.reshape(bboxes, (1, -1, 4))
        classes = np.reshape(classes, (1, -1))
        scores = np.reshape(scores, (1, -1))
        return (bboxes, classes, scores), labels[0]

In [7]:
from neural_compressor import options
from neural_compressor.experimental import Quantization, common
options.onnxrt.graph_optimization.level = 'ENABLE_BASIC'

model = onnx.load("models/MaskRCNN-12.onnx")
dataloader = Dataloader("data/")

quantize = Quantization("mask_rcnn.yaml")
quantize.model = common.Model(model)
quantize.eval_dataloader = dataloader
quantize.calib_dataloader = dataloader
quantize.postprocess = common.Postprocess(Post)
q_model = quantize()
q_model.save("models/quantized/mask_rcnn.onnx")

2022-12-19 00:05:47 [INFO] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-12-19 00:05:47 [INFO] NumExpr defaulting to 8 threads.


loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


/tmp/ipykernel_442278/2091554627.py:53: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  image = image.resize((int(ratio * image.size[0]), int(ratio * image.size[1])), Image.BILINEAR)


: 

: 

As you can see above memory requirement is large so Quantizing Model doesn't work locally on a 32 GB Memory PC